In [1]:
from choice_model import ChoiceModel
import networkx as nx
import os

In [2]:
folder = 'sim_data'
base_url = 'http://localhost:11434'
remove_type = None

if not os.path.exists(f'experiments/{folder}'):
    os.makedirs(f'experiments/{folder}')


def predict(period, remove_type=None, k1=5, k2=1, top_k=10):
    model = ChoiceModel(base_url=base_url)
    if period == 0:
        test_graph = nx.DiGraph()
    else:
        prev_period_file = f'experiments/{folder}/{folder}_{period-1}.pkl'
        new_model = ChoiceModel(graph_path=prev_period_file, base_url=base_url)
        test_graph = new_model.graph.copy()

    graph = model.get_period(period)
    new_nodes = [n for n in graph.nodes if n not in list(test_graph.nodes)]
    new_nodes = [n for n in new_nodes if graph.nodes[n]['type'] != remove_type]

    print("begin simulation", len(test_graph.nodes), len(test_graph.edges))
    test_graph, failed_nodes = model.predict_links(test_graph, new_nodes, file_name=f"experiments/{folder}/{folder}_{period}.pkl",
                                                   choice_type=None, k1=k1, k2=k2, top_k=top_k, period=period)

    if len(failed_nodes) > 0:
        print("retrying failed nodes")
        test_graph, _ = model.predict_links(test_graph, failed_nodes, file_name=f"experiments/{folder}/{folder}_{period}.pkl",
                                            choice_type=None, k1=k1, k2=k2, top_k=top_k, period=period)

    test_graph.remove_edges_from(nx.selfloop_edges(test_graph))
    model.save_graph(
        file_name=f"experiments/{folder}/{folder}_{period}.pkl", graph=test_graph)
    new_model = ChoiceModel(
        graph_path=f"experiments/{folder}/{folder}_{period}.pkl")
    new_model.visualize(
        file_name=f"experiments/{folder}/{folder}_{period}.html")
    print("end simulation", len(test_graph.nodes), len(test_graph.edges))
    CPR, ACI, MCP = new_model.evaluate(test_graph)
    print(f"Period {period} CPR: {CPR} ACI: {ACI} MCP: {MCP}")

In [ ]:
for i in range(1, 10):
    predict(i, k1=5, k2=1, top_k=10, remove_type=remove_type)